# 2024C1: Plotting SST1 RSoXS Data

## Imports

In [ ]:
# !pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm
import dask.array as da
# from tiled.client import from_profile, from_uri
import subprocess


print(f'Using PyHyperScattering Version: {phs.__version__}')

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

## Define paths and functions

In [ ]:
def make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==45:
        para_DA = sliced_DA.rsoxs.slice_chi(-135, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(135, chi_width=(chi_width/2))        
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

In [ ]:
## Define directory paths
# userPath = pathlib.Path('/nsls2/users/alevin')
propPath = pathlib.Path('/nsls2/data/sst/proposals/2024-1/pass-313412')

outPath = propPath.joinpath('processed_data')
plotsPath = outPath.joinpath('rsoxs_plots')
# jsonPath = outPath.joinpath('local_config')
maskPath = outPath.joinpath('masks')
zarrsPath = outPath.joinpath('rsoxs_zarrs')


# propPath = pathlib.Path('/nsls2/data/sst/proposals/2022-2/pass-309180')
# zarrsPath = propPath.joinpath('zarr_datasets_jul2022')
# outPath = propPath.joinpath('processed_data')
# plotsPath = outPath.joinpath('prsoxs_plots')

## Load DataArrays 

In [ ]:
[f.name for f in zarrsPath.glob('*')]

In [ ]:
# load dictionary of rsoxs datasets
rsoxs_datasets = {}
for key in ['cartesian_1180', 'polar_1180']:
# for key in ['polar_carbon']:
    key_start = key.split('_')[0]
    key_end = key.split('_')[1]        
    zarrPath = list(zarrsPath.glob(f'{key_start}*{key_end}.zarr'))[0]
    rsoxs_datasets[key] = xr.open_zarr(zarrPath).sortby('sample_name')
    
    # Compute any dask coordiantes
    for coord_name, coord_data in rsoxs_datasets[key].coords.items():
        if isinstance(coord_data.data, da.Array):
            rsoxs_datasets[key].coords[coord_name] = coord_data.compute()

In [ ]:
rsoxs_datasets = {}
rsoxs_datasets['polar_carbon'] = xr.open_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_v2.zarr'))
rsoxs_datasets['cartesian_carbon'] = xr.open_zarr(zarrsPath.joinpath('cartesian_rsoxs_carbon_v1.zarr'))
rsoxs_datasets['polar_carbon_reverse'] = xr.open_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_reverse_v1.zarr'))
rsoxs_datasets['cartesian_carbon_reverse'] = xr.open_zarr(zarrsPath.joinpath('cartesian_rsoxs_carbon_reverse_v1.zarr'))
rsoxs_datasets['polar_carbon_tilted'] = xr.open_zarr(zarrsPath.joinpath('polar_rsoxs_carbon_tilted_v1.zarr'))
rsoxs_datasets['cartesian_carbon_tilted'] = xr.open_zarr(zarrsPath.joinpath('cartesian_rsoxs_carbon_tilted_v1.zarr'))

# Compute any dask coordiantes
for key in rsoxs_datasets.keys():
    for coord_name, coord_data in rsoxs_datasets[key].coords.items():
        if isinstance(coord_data.data, da.Array):
            rsoxs_datasets[key].coords[coord_name] = coord_data.compute()

In [ ]:
DS = rsoxs_datasets['cartesian_carbon_reverse']

In [ ]:
DS

In [ ]:
# sorted([f.name for f in zarrsPath.glob('TRMSN*nexafs*')])

In [ ]:
# trmsn_4angle = xr.open_zarr(zarrsPath.joinpath('TRMSN_nexafs_carbon_day1.zarr'))
# trmsn_3angle = xr.open_zarr(zarrsPath.joinpath('TRMSN_nexafs_carbon_3angle.zarr'))

# trmsn_90_part1 = trmsn_4angle.sel(theta=90)
# trmsn_90_part2 = trmsn_3angle.sel(theta=90)

# trmsn90 = xr.concat([trmsn_90_part1, trmsn_90_part2], dim='sample_name')
# trmsn90

In [ ]:
# polar_netcdf_paths = str(zarrsPath.joinpath('polar_rsoxs_carbon_ncs')) + '/*.nc'
# polar_netcdf_paths

In [ ]:
# # rsoxs_datasets = {}
# # rsoxs_datasets['polar_carbon'] = xr.open_mfdataset(polar_netcdf_paths, parallel=True)
# rsoxs_datasets['polar_carbon'] = xr.open_mfdataset(polar_netcdf_paths)
# rsoxs_datasets['polar_carbon']

In [ ]:
trmsn90_sample_names = []
for sample_name in trmsn90.sample_name.values:
    trmsn90_sample_names.append(sample_name.replace('_rot', ''))
    
display(trmsn90_sample_names)

In [ ]:
bad_trmsn_samples = ['Y6_CBCN', 'Y6BO_CBCN', 'Y6BO_CFCN','Y7_CB_2000', 'Y7_CB_2000_1', 'Y7_CBCN', 'Y7_CFCN', 
                     'Y7BO_CFCN', 'BareSiN_1p5mm', 'BareSiN_1mm', 'BareSiN_1mm_1', 'BareSiN_1mm_2']

In [ ]:
selected_samples = [sample_name for sample_name in rsoxs_datasets['polar_carbon'].sample_name.values if sample_name not in bad_trmsn_samples]
selected_samples

In [ ]:
for sample_name in selected_samples:
    print(sample_name)
    sample_name = sample_name.replace('_1', '')
    print(sample_name in trmsn90_sample_names)
    print('\n')

In [ ]:
trmsn90_DA = trmsn90['full_corr_trmsn']
# pre_edge_mean = trmsn90_DA.sel(energy=slice(270,280)).mean('energy')
# trmsn90_DA = (trmsn90_DA - pre_edge_mean) * -1

trmsn90_DA

In [ ]:
for sample_name in trmsn90_DA.sample_name.values:
    trmsn90_DA.sel(sample_name=sample_name).plot()
    plt.show()

In [ ]:
polar_DS = rsoxs_datasets['polar_carbon']
trmsn90_DA = trmsn90_DA.interp({'energy': polar_DS.energy.values})
# polar_DS

In [ ]:
polar_DS['corr_intensity']

In [ ]:
trmsn90_samp_DA.drop_vars(['sample_id', 'sample_name', 'theta', 'scan_id'])

In [ ]:
fl_bkg_DA

In [ ]:
fl_bkgs = []
trmsn90_bkgs = []

for sample_name in tqdm(selected_samples):
    polar_samp_DA = polar_DS['corr_intensity'].sel(sample_name=sample_name)
    fl_bkg_DA = 1 * polar_samp_DA.sel(polarization=0,q=slice(0.02, 0.06)).mean('chi').integrate('q')
    fl_bkgs.append(fl_bkg_DA)
    # fl_bkg_DA.plot()
    # plt.show()
    
    trmsn90_samp_DA = trmsn90_DA.sel(sample_name=(sample_name.replace('_1','')+'_rot')).drop_vars(['sample_id', 'sample_name', 'theta', 'scan_id'])
    trmsn90_samp_DA = trmsn90_samp_DA.expand_dims({'sample_name':[sample_name]})
    trmsn90_bkgs.append(trmsn90_samp_DA)
#     trmsn90_samp_DA.plot()
#     plt.show()
    
#     plt.close('all')
        

In [ ]:
# DS['alo_sub_raw_intensity'] = DS['raw_intensity'] - DS['raw_intensity'].sel(sample_name='BareAlO')
# DS['alo_sub_corr_intensity'] = DS['corr_intensity'] - DS['corr_intensity'].sel(sample_name='BareAlO')

cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

## 1180 eV non-resonant single images

### Cart 2D

In [ ]:
# Select cartesian dataset, 1180
edge = '1180'
DS = rsoxs_datasets[f'cartesian_{edge}'].copy()
bcx = DS['raw_intensity'].beamcenter_x
bcy = DS['raw_intensity'].beamcenter_y


# Select Plotting Parameters
# energy = 285
pix_size = 500
pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Pol = {pol}°'):
        intensity_type = 'raw'
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
        energy = int(DA.energy.values)
        cmin = float(DA.compute().quantile(0.0001))  # good for 1180
        cmax = float(DA.compute().quantile(0.995))  # good for 1180

        # Plot
        # sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy,method='nearest')
        sliced_DA = DA.sel(polarization=pol).sel(energy=energy,method='nearest')
        # print(f'cmin={cmin}, cmax={cmax}')
        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='qx', y='qy', cmap=cmap, norm=plt.Normalize(cmin,cmax))
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
        ax.colorbar.set_label('Raw Detector Intensity [arb. units]', rotation=270, labelpad=12)
        
        # savePath = plotsPath.joinpath('rsoxs_1180/cart_2D')
        # ax.figure.savefig(savePath.joinpath(f'{sample_name}_{edge}_pol{pol}deg.png'), dpi=120)
        plt.show()
        plt.close('all')

### Polar 2D

In [ ]:
# Select polar dataset
edge = '1180'
DS = rsoxs_datasets[f'polar_{edge}'].copy()

# Select DataArray
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Pol = {pol}°'):
        intensity_type = 'raw'
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
        energy = int(DA.energy.values)
        cmin = float(DA.compute().quantile(0.0001))  # good for 1180
        cmax = float(DA.compute().quantile(0.995))  # good for 1180

        # Plot
        # sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy,method='nearest')
        sliced_DA = DA.sel(polarization=pol).sel(energy=energy,method='nearest')
        # print(f'cmin={cmin}, cmax={cmax}')
        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=plt.Normalize(cmin,cmax))
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_r$ [$Å^{-1}$]', ylabel='$\chi$ [°]')
        ax.colorbar.set_label('Raw Detector Intensity [arb. units]', rotation=270, labelpad=12)
        
        # savePath = plotsPath.joinpath('rsoxs_1180/polar_2D')
        # ax.figure.savefig(savePath.joinpath(f'{sample_name}_{edge}_pol{pol}deg.png'), dpi=120)
        plt.show()
        plt.close('all')

### Qr linecut

In [ ]:
# Select polar dataset, to qr linecuts: para, perp, and full
edge = '1180'
chi_width = 90
q_slice = slice(0.1,0.4)
DS = rsoxs_datasets[f'polar_{edge}'].copy()

# Select DataArray
for sample_name in tqdm(DS.sample_name.values):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, 'raw', pol, (None,None), chi_width) 
        para_DA = para_DA.compute()
        perp_DA = perp_DA.compute()
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
    
    energy = int(DA.energy.values)
    # cmin = float(DA.compute().quantile(0.0001))  # good for 1180
    # cmax = float(DA.compute().quantile(0.995))  # good for 1180

    # Plot
    regions = ['para', 'perp', 'full']
    colors = plt.cm.Dark2(np.linspace(0, 1, 8))

    fig, ax1 = plt.subplots(figsize=(6,3.5), tight_layout=True, dpi=120)
    ax2 = ax1.twinx()
    ax3 = ax1.twinx()
    

    # for j, energy in enumerate(energies):
    p2, = (para_DA.sel(q=q_slice, energy=energy).mean('chi')
     .plot.line(ax=ax2, color=colors[1], yscale='linear', xscale='linear', label='Para'))
    p3, = (perp_DA.sel(q=q_slice, energy=energy).mean('chi')
     .plot.line(ax=ax3, color=colors[2], yscale='linear', xscale='linear', label='Perp'))
    p1, = (((para_DA.mean('chi') + perp_DA.mean('chi')).sel(q=q_slice, energy=energy)/2)
     .plot.line(ax=ax1, color=colors[0], yscale='linear', xscale='linear', label='Full'))

    fig.suptitle(f'I vs Q linecuts: {sample_name}', y=0.93, x=0.43)

    ax1.set_title('$\chi_{width}$ =' + f' {chi_width}°, averaged polarizations')
    ax1.set(ylabel='Full $\chi$ intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax2.set(ylabel='Para $\chi$ intensity [arb. units]', title=None)
    ax3.set(ylabel='Perp $\chi$ intensity [arb. units]', title=None)

    # fig.legend(title='$\chi$ region',borderaxespad=4.5)
    
    ax1.tick_params(axis='y', colors=p1.get_color())
    ax1.yaxis.label.set_color(p1.get_color())
    ax2.yaxis.label.set_color(p2.get_color())
    ax3.yaxis.label.set_color(p3.get_color())
    
    ax2.tick_params(axis='y', colors=p2.get_color())
    ax2.spines['right'].set_position(('outward', 0))
    ax3.spines['right'].set_edgecolor(p2.get_color())
    
    ax3.tick_params(axis='y', colors=p3.get_color())
    ax3.spines['right'].set_position(('outward', 60))
    ax3.spines['right'].set_edgecolor(p3.get_color())
    
    lines= [p1,p2,p3]
    ax1.legend(title='$\chi$ regions', handles=lines, labels=[l.get_label() for l in lines])

    # savePath = plotsPath.joinpath('rsoxs_1180/qr_linecuts_v1')
    # fig.savefig(savePath.joinpath(f'{sample_name}_{edge}.png'), dpi=120)

    plt.show()
    plt.close('all')

### Chi linecut

In [ ]:
# Select polar dataset, to chi linecuts: 
edge = '1180'
chi_slice = slice(None,None)
q_slice_1 = slice(0.15, 0.22)
q_slice_2 = slice(0.23, 0.38)
DS = rsoxs_datasets[f'polar_{edge}'].copy()

# Select DataArray
for sample_name in tqdm(DS.sample_name.values):
    pol_DAs = []
    for pol in [0, 90]:
        pol_DAs.append(DS['raw_intensity'].sel(sample_name=sample_name, polarization=pol).squeeze())

    # energy = int(DA.energy.values)
    # cmin = float(DA.compute().quantile(0.0001))  # good for 1180
    # cmax = float(DA.compute().quantile(0.995))  # good for 1180

    # Plot
    colors = plt.cm.Dark2(np.linspace(0, 1, 8))
    fig, (left_ax1, right_ax1) = plt.subplots(ncols=2, figsize=(7.5,3.5), tight_layout=True, dpi=120)
    left_ax2 = left_ax1.twinx()
    right_ax2 = right_ax1.twinx()

    p1, = (pol_DAs[0].sel(q=q_slice_1, chi=chi_slice).integrate('q')
     .plot.line(ax=left_ax1, color='#1054E1', yscale='linear', xscale='linear', label='0'))
    p2, = (pol_DAs[1].sel(q=q_slice_1, chi=chi_slice).integrate('q')
     .plot.line(ax=left_ax2, color='#E19C10', yscale='linear', xscale='linear', label='90'))
    
    p3, = (pol_DAs[0].sel(q=q_slice_2, chi=chi_slice).integrate('q')
     .plot.line(ax=right_ax1, color='#1054E1', yscale='linear', xscale='linear', label='0'))
    p4, = (pol_DAs[1].sel(q=q_slice_2, chi=chi_slice).integrate('q')
     .plot.line(ax=right_ax2, color='#E19C10', yscale='linear', xscale='linear', label='90'))

    fig.suptitle(f'I vs Chi linecuts: {sample_name}', y=0.93, x=0.5)

    left_ax1.xaxis.set_major_locator(MultipleLocator(90))
    left_ax1.xaxis.set_minor_locator(MultipleLocator(45))
    left_ax1.xaxis.grid(True, which='both')
                                     
    right_ax1.xaxis.set_major_locator(MultipleLocator(90))
    right_ax1.xaxis.set_minor_locator(MultipleLocator(45))
    right_ax1.xaxis.grid(True, which='both')    

    left_ax1.set_title(f'Q slice = ({q_slice_1.start}, {q_slice_1.stop})' + ' Å$^{-1}$')
    left_ax1.set(ylabel='Pol = 0° Intensity [arb. units]', xlabel='$\chi$ [°]')
    left_ax2.set(ylabel='Pol = 90° Intensity [arb. units]', title=None)

    right_ax1.set_title(f'Q slice = ({q_slice_2.start}, {q_slice_2.stop})' + ' Å$^{-1}$')
    right_ax1.set(ylabel='Pol = 0° Intensity [arb. units]', xlabel='$\chi$ [°]')
    right_ax2.set(ylabel='Pol = 90° Intensity [arb. units]', title=None)
    
#     # fig.legend(title='$\chi$ region',borderaxespad=4.5)

    left_ax1.tick_params(axis='y', colors=p1.get_color())
    left_ax2.tick_params(axis='y', colors=p2.get_color())
    left_ax1.yaxis.label.set_color(p1.get_color())
    left_ax2.yaxis.label.set_color(p2.get_color())

    right_ax1.tick_params(axis='y', colors=p3.get_color())
    right_ax2.tick_params(axis='y', colors=p4.get_color())
    right_ax1.yaxis.label.set_color(p3.get_color())
    right_ax2.yaxis.label.set_color(p4.get_color())

    # lines= [p1,p2]
    # # left_ax1.legend(title='$\chi$ regions', handles=lines, labels=[l.get_label() for l in lines])
    # right_ax1.legend(title='$\chi$ regions', handles=lines, labels=[l.get_label() for l in lines])

    # savePath = plotsPath.joinpath('rsoxs_1180/chi_linecuts_v1')
    # fig.savefig(savePath.joinpath(f'{sample_name}_{edge}.png'), dpi=120)

    plt.show()
    plt.close('all')

## Intensity Plots

### 1. Detector movies

In [ ]:
%matplotlib inline

In [ ]:
plt.close('all')

In [ ]:
# Parameters
edge = 'carbon'
# DS = rsoxs_datasets[f'cartesian_{edge}']
DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
intensity_type = 'corr'
pol = 0
energy = 285.2

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
# selected_samples = ['PM6_5CN-CB']

for sample_name in DS.sample_name.values:
# for sample_name in selected_samples:
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    DA = DA.where(DA>0, 10)
    # Make & customize plot
    sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})

    cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.03))
    cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))
    
    # cmin = cmin if cmin > 1e8 else 1e8

    ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
    ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
    ax.figure.set_tight_layout(True)   
    ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
    ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)

    plt.show()
    plt.close('all')

In [ ]:
cmin

In [ ]:
plotsPath

In [ ]:
# Select Dataset
edge = 'carbon'
DS = rsoxs_datasets[f'cartesian_{edge}_reverse']
# DS = rsoxs_datasets[f'cartesian_{edge}_tilted']

# bcx = DS['raw_intensity'].beamcenter_x
# bcy = DS['raw_intensity'].beamcenter_y

# Select Plotting Parameters
# pol = 90
# pix_size = 500
# pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
# pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
intensity_type = 'corr'

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
# selected_samples = ['PM6_5CN-CB']
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
    # for sample_name in tqdm(selected_samples, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        savePath = plotsPath.joinpath('rsoxs_carbon_reverse/detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '16',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '22',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} movie')):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})
            
#             cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.05))
#             cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))

#             cmin = cmin if cmin > 1e8 else 1e8
            
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)

            # Save first frame (or specific energy value)
            if energy == 285.2:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")


In [ ]:
plotsPath

### 2. ISI Plots

In [ ]:
# Select polar dataset, to ISI plots, polarizations 0 & 90 averaged
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}_reverse'].copy()
# DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

# DS = DS.rename_vars({'raw_intensity':'corr_intensity'})

chi_width = 90
q_slice = slice(0.009,0.08)
e_slice = slice(282, 295)

# make selection
intensity_type = 'corr'

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr

    # slice ISI data
    para_ISI = para_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')
    perp_ISI = perp_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')

    # plot
    fig, ax = plt.subplots(figsize=(6,4), dpi=120)
    ax.xaxis.set_minor_locator(MultipleLocator(1))

    para_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\parallel$', yscale='log')
    perp_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\perp$', yscale='log')
    fig.suptitle(f'ISI: {sample_name}', fontsize=14, x=0.55)
    ax.set(title=f'Pol = {pol}°, chi width = {chi_width}°, Q = ({q_slice.start}, {q_slice.stop}) ' + 'Å$^{-1}$', 
           xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
    ax.legend(loc='upper right', fontsize=14)
    ax.grid(axis='x', which='both')
    plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)
        
    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ISIs_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chi{chi_width}deg_q{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
plotsPath

In [ ]:
# Select polar dataset, to ISI plots, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

# DS = DS.rename_vars({'raw_intensity':'corr_intensity'})

chi_width = 90
q_slice = slice(0.009,0.08)
e_slice = slice(282, 295)

# make selection
intensity_type = 'corr'

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').sel(q=q_slice).mean('chi').mean('q')

        # plot
        fig, ax = plt.subplots(figsize=(6,4), dpi=120)
        ax.xaxis.set_minor_locator(MultipleLocator(1))

        para_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\parallel$', yscale='log')
        perp_ISI.sel(energy=e_slice).plot.line(ax=ax, label='$\perp$', yscale='log')
        fig.suptitle(f'ISI: {sample_name}', fontsize=14, x=0.55)
        ax.set(title=f'Pol = {pol}°, chi width = {chi_width}°, Q = ({q_slice.start}, {q_slice.stop}) ' + 'Å$^{-1}$', 
               xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
        ax.legend(loc='upper right', fontsize=14)
        ax.grid(axis='x', which='both')
        plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ISIs_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
            f'{sample_name}_{intensity_type}_chi{chi_width}deg_q{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']
# selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']

### 3. 2D Linecut Maps

In [ ]:
# Select polar dataset, to I cuts, averaged 0 & 90 deg polarization
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
# DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()
DS = rsoxs_datasets[f'polar_{edge}_reverse'].copy()

chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
intensity_type = 'corr'
pol = 0

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr
                
    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(11,5))
    # cmin = 1e8
    # cmax = 1e11

    para_slice = para_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice = perp_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    cmin = perp_slice.compute().quantile(0.01)
    cmax = para_slice.compute().quantile(0.995)

    para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

    fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

    fig.set(tight_layout=True)

    axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
    axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/I_maps_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{e_slice.start}-{e_slice.stop}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# Select polar dataset, to I cuts, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}'].copy()
DS = rsoxs_datasets[f'polar_{edge}_tilted'].copy()

chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
intensity_type = 'corr'
pol = 0

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # Plot
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_slice = para_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice = perp_DA.mean('chi').sel(q=q_slice, energy=e_slice)  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        cmin = perp_slice.compute().quantile(0.01)
        cmax = para_slice.compute().quantile(0.995)

        para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

        fig.set(tight_layout=True)

        axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/I_maps_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
            f'{sample_name}_{e_slice.start}-{e_slice.stop}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

In [ ]:
# a = para_EvsQ_avg.hvplot.line(groupby="energy", loglog=True)
# a * perp_EvsQ_avg.hvplot.line(groupby="energy")

### 4. Meaned 1D intensity lineplots, selected energies

In [ ]:
# trmsn90_bkgs_DA = xr.concat(trmsn90_bkgs, dim='sample_name')
# trmsn90_bkgs_DA

In [ ]:
# for sample_name in trmsn90_bkgs_DA.sample_name.values:
#     trmsn90_bkgs_DA.sel(sample_name=sample_name).plot()
#     plt.show()

In [ ]:
# fl_bkgs_DA = xr.concat(fl_bkgs, dim='sample_name')
# fl_bkgs_DA

In [ ]:
# trmsn90_corr.plot()

In [ ]:
# manual_scale_factors_v4 = {
#     'PM6_CB_3000': 8.4,
#     'PM6_CBCN': 11,
#     'PM6_CF_3000': 6,
#     'PM6_CFCN': 10.5,
#     'PM6-Y6_CB': 6.3,
#     'PM6-Y6_CBCN': 4.5,
#     'PM6-Y6_CF': 6,
#     'PM6-Y6_CFCN': 11.5,
#     'PM6-Y6BO_CB': 7.5,
#     'PM6-Y6BO_CF': 5,
#     'PM6-Y6BO_CFCN': 12.8,
#     'PM6-Y7_CB': 6.5,
#     'PM6-Y7_CBCN': 8.5,
#     'PM6-Y7_CF': 4.5,
#     'PM6-Y7_CFCN': 12.8,
#     'PM6-Y7BO_CB': 5,
#     'PM6-Y7BO_CBCN': 8.2,
#     'PM6-Y7BO_CF': 5,
#     'PM6-Y7BO_CFCN': 10,
#     'Y6_CB_2000_1': 5,
#     'Y6_CB_2000': 6,
#     'Y6_CB_2500': 6,
#     'Y6_CB_3000': 6,
#     'Y6_CF_2000': 18,
#     'Y6_CF_2500': 20,
# }

# len(manual_scale_factors_v4)

In [ ]:
# make selection, averaged polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'corr'
qlims = (0.009, 0.08)
chi_width = 90

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
#     # Fl correction?
#     scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
#     manual_scale_factor = manual_scale_factors_v4[sample_name]
#     trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
#     para_DA = para_DA - trmsn90_corr
#     perp_DA = perp_DA - trmsn90_corr
        
    # Plot
    energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data

    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))

    fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

    qmin = qlims[0]
    qmax = qlims[1]

    for j, energy in enumerate(energies):
        (para_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
        (perp_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

    fig.suptitle(f'IvsQ, {pol}° pol, 90° chi width: {sample_name}', x=0.47)
    # fig.suptitle(f'IvsQ ($Fl$ Corr (C={manual_scale_factor})), {pol}° pol, 90° chi width: {sample_name}', x=0.47)

    axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[0].set_ylim(bottom=1e8)
    axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[1].set_ylim(bottom=1e8)
    axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/I_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath(
        f'{sample_name}_chiWidth-{chi_width}deg_q{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# Drsoxs_datasets rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
qlims = (0.009, 0.08)
chi_width = 90

# for sample_name in tqdm(filtered_selected_samples):
for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, qlims, chi_width) 

        # Plot
        energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data

        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))

        fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

        qmin = qlims[0]
        qmax = qlims[1]

        for j, energy in enumerate(energies):
            (para_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
             .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
            (perp_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
             .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

        fig.suptitle(f'IvsQ, {pol}° pol, 90° chi width: {sample_name}', x=0.47)
        # fig.suptitle(f'IvsQ ($Fl$ Corr (C={manual_scale_factor})), {pol}° pol, 90° chi width: {sample_name}', x=0.47)

        axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[0].set_ylim(bottom=1e8)
        axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
        axs[1].set_ylim(bottom=1e8)
        axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/I_cuts_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath(
            f'{sample_name}_chiWidth-{chi_width}deg_q{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

## Anisotropy Ratio Plots

### 1. 2D AR Maps

In [ ]:
# make selection, averaged polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):  
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot
    vlim = 0.4
    ax = ar_DA.sel(energy=e_slice).plot.pcolormesh(figsize=(8,5), norm=plt.Normalize(-vlim, vlim))
    # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

    ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
    ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
    ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ar_maps_v1')
    savePath.mkdir(exist_ok=True)
    ax.figure.savefig(savePath.joinpath( 
        f'vset_{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{e_slice.start}-{e_slice.stop}_pol{pol}deg.png'), 
                      dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):  
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 


        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot
        vlim = 0.5
        ax = ar_DA.sel(energy=e_slice).plot.pcolormesh(figsize=(8,5), norm=plt.Normalize(-vlim, vlim))
        # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ar_maps_v1')
        savePath.mkdir(exist_ok=True)
        ax.figure.savefig(savePath.joinpath( 
            f'vset_{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_energy{e_slice.start}-{e_slice.stop}_pol{pol}deg.png'), 
                          dpi=120)

        # plt.show()
        plt.close('all')

### 2. 1D meaned AR linecuts

In [ ]:
# make selection
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_reverse']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.008, None)
e_slice = slice(282, 292)

selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
                    'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
                    'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 

for sample_name in tqdm(DS.sample_name.values[:]):
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(energies):
        (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

    fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.legend(title='Energy [eV]', loc=(1.03,0.02))

    savePath = plotsPath.joinpath('rsoxs_carbon_reverse/ar_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# make selection, individual polarizations
edge = 'carbon'
# DS = rsoxs_datasets[f'polar_{edge}']
DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.009, 0.08)
e_slice = slice(282, 292)

for sample_name in tqdm(DS.sample_name.values[:]):
    for pol in [0, 45, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 

        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        # Plot   
        energies = para_DA.energy.sel(energy=[260, 275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 338.75], method='nearest').data
        cmap = plt.cm.turbo.copy()
        colors = cmap(np.linspace(0, 1, len(energies)))        

        # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
        # cax = ax.inset_axes([1.03, 0, 0.03, 1])
        # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        # cbar.set_label(label=f'Time [seconds]', labelpad=14)
        # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

        fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

        for j, energy in enumerate(energies):
            (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log'))

        fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46, y=0.95)

        # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
        ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')

        ax.legend(title='Energy [eV]', loc=(1.03,0.02))

        savePath = plotsPath.joinpath('rsoxs_carbon_tilted/ar_cuts_v1')
        savePath.mkdir(exist_ok=True)
        fig.savefig(savePath.joinpath( 
            f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

        # plt.show()
        plt.close('all')

### Para - perp log scale intensity

In [ ]:
%matplotlib inline

In [ ]:
plotsPath

In [ ]:
porod_slice

In [ ]:
# make selection
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.01, None)
e_slice = slice(282, 292)
porod_slices = []
# porod_slices = [slice(0.0107, 0.0138), # Calculate 1 degree polynomial (straigh line) through these regions
#                 slice(0.0107, 0.0240),
#                 slice(0.0250, 0.0500),
#                 slice(0.0500, 0.0800)]

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
selected_samples = ['PM6_5CN-CB', 'PM6_CF']

plt.close('all')

all_slopes = {}
# for sample_name in tqdm(DS.sample_name.values[:]):
for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    slopes = {}
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    # ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    ar_DA = np.abs(para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[260, 283, 284.4, 284.8, 285.2, 286.2, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(tqdm(energies, desc='Plotting & fitting slopes per energy')):
        sel_DA = ar_DA.sel(energy=energy)
        sel_DA.plot.line(ax=ax, color=colors[j], label=energy, xscale='log', yscale='log',
                         linestyle='None', marker='.', markersize=3)
        
        porod_slopes = {}
        for porod_slice in porod_slices:
            m, b = np.polyfit(np.log(sel_DA.sel(q=porod_slice).q.data), np.log(sel_DA.sel(q=porod_slice).data.compute()), deg=1)
            # display(porod_slice, m)       
            porod_slopes[f'{porod_slice.start}-{porod_slice.stop}'] = np.round(m, 2)
            
        slopes[energy] = porod_slopes
        
    all_slopes[sample_name] = slopes
    fig.suptitle(f'$\parallel$ - $\perp$ Intensity Absolute Difference: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(1e8, None), ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(None, None), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.axes.grid(visible=True, which='both', axis='x')
    legend = ax.legend(title='Energy [eV]', loc=(1.03,0.23))
    for handle in legend.legend_handles:
        handle.set_markersize(15)
    
    # savePath = plotsPath.joinpath('rsoxs_carbon/para-perp_cuts_v3')
    # savePath.mkdir(exist_ok=True)
    # fig.savefig(savePath.joinpath( 
    #     f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')
    
# # Save all_slopes.json in same save folder
# json_data = json.dumps(all_slopes)
# with open(str(savePath.joinpath('all_slopes.json')), 'w') as f:
#     f.write(json_data)

In [ ]:
all_slopes

### Load solution SAXS PM6 data text files

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
# Check files
saxsPath = pathlib.Path('/nsls2/data/sst/proposals/2023-3/pass-313412/processed_data/prsoxs_plots/carbon/PM6_solution_saxs_txts')
# saxsPath = outPath.joinpath('prsoxs_plots/carbon/PM6_solution_saxs_txts')
display([f.name for f in sorted(saxsPath.glob('*'))[:3]])
display([f.name for f in sorted(saxsPath.glob('*'))[3:]])

In [ ]:
# plt.close('all')
# sigmas = [1, 2, 3, 4]
# colors = plt.cm.viridis(np.linspace(0,0.85,len(sigmas)))
# for arr in [pm6_cb, pm6_cf, pm6_cn]:
#     fig, ax = plt.subplots()
#     for i, sigma in enumerate(sigmas[::1]):
#         ax.plot(gaussian_filter1d(arr[:,1], sigma), color=colors[i], label=sigma)
#         ax.set(xscale='log', yscale='log')
#         ax.legend(title='sigma')

#     # plt.show()
#     # plt.close('all')

In [ ]:
# Load data into numpy arrays
pm6_cb, pm6_cf, pm6_cn = [np.loadtxt(f) for f in sorted(saxsPath.glob('*'))[:3]]
y6_cn, y6_to, y7_cb, y7_cn = [np.loadtxt(f) for f in sorted(saxsPath.glob('*'))[3:]]

# Smooth saxs with gaussian filter:
sigma = 3  # In pixels
pm6_cb[:,1], pm6_cf[:,1], pm6_cn[:,1] = [gaussian_filter1d(arr[:,1], sigma) for arr in [pm6_cb, pm6_cf, pm6_cn]]
y6_cn[:,1], y6_to[:,1], y7_cb[:,1], y7_cn[:,1] = [gaussian_filter1d(arr[:,1], sigma) for arr in [y6_cn, y6_to, y7_cb, y7_cn]]

# Below should really just be a loop... made sense for 3 samples
# Put numpy arrays into DataArrays
pm6_cb_DA = xr.DataArray(data=pm6_cb[:,1], dims=['q'], coords={'q':pm6_cb[:,0], 'err':('q', pm6_cb[:,2])})
pm6_cf_DA = xr.DataArray(data=pm6_cf[:,1], dims=['q'], coords={'q':pm6_cf[:,0], 'err':('q', pm6_cf[:,2])})
pm6_cn_DA = xr.DataArray(data=pm6_cn[:,1], dims=['q'], coords={'q':pm6_cn[:,0], 'err':('q', pm6_cn[:,2])})                                              
y6_cn_DA = xr.DataArray(data=y6_cn[:,1], dims=['q'], coords={'q':y6_cn[:,0], 'err':('q', y6_cn[:,2])})
y6_to_DA = xr.DataArray(data=y6_to[:,1], dims=['q'], coords={'q':y6_to[:,0], 'err':('q', y6_to[:,2])})
y7_cb_DA = xr.DataArray(data=y7_cb[:,1], dims=['q'], coords={'q':y7_cb[:,0], 'err':('q', y7_cb[:,2])})
y7_cn_DA = xr.DataArray(data=y7_cn[:,1], dims=['q'], coords={'q':y7_cn[:,0], 'err':('q', y7_cn[:,2])})
                                                  
# Move DataArrays into Dataset
saxs_DS = xr.Dataset(attrs={'name':'PM6_Solution_SAXS'})
saxs_DS['PM6_CB'] = pm6_cb_DA
saxs_DS['PM6_CF'] = pm6_cf_DA
saxs_DS['PM6_CN'] = pm6_cn_DA                                            
saxs_DS['Y6_CN'] = y6_cn_DA
saxs_DS['Y6_TO'] = y6_to_DA
saxs_DS['Y7_CB'] = y7_cb_DA
saxs_DS['Y7_CN'] = y7_cn_DA

saxs_DS

In [ ]:
# # Ensure q values are equal between datasets, else interpolation needed
# print(np.array_equal(pm6_cb[:,0], pm6_cf[:,0]))
# print(np.array_equal(pm6_cf[:,0], pm6_cn[:,0]))

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['PM6_CB', 'PM6_CF', 'PM6_CN']:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

In [ ]:
%matplotlib widget

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['Y6_CN', 'Y7_CB', 'Y7_CN']:
# for sample in ['Y7_CB', 'Y7_CN']:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

### Plot para-perp P-RSoXS + solution SAXS overlays

In [ ]:
%matplotlib widget

In [ ]:
sn = {
    'PM6_CF': 'PM6, CF',
    'PM6_5CN-CF': 'PM6, CF 5%CN',
    'PM6_5CN-CB': 'PM6, CB 5%CN'
}

In [ ]:
# make selection
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.01, None)
# e_slice = slice(282, 292)
energy = 285.2
# energy = 284.8
# energies = [260, 283, 284.4, 284.8, 285.2, 286.2, 338.75]
# shift_qval = 0.0104  # value by which to look to shift data, divide rsoxs by this value
shift_qval = 0.02  # value by which to look to shift data, divide rsoxs by this value

# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB', 
#                     'PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF', 
#                     'PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB'] 
selected_samples = ['PM6_CF', 'PM6_5CN-CB']
# selected_samples = ['PM6_5CN-CF']
# selected_samples = ['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB']
# selected_samples = ['PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF']
# selected_samples = ['PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB']

# selected_samples = ['Y6_CF', 'Y6_p5CN-CF', 
#                     'Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB', 
#                     'Y6_CB', 'Y6_p5CN-CB',
#                     'Y6BO_CF', 'Y6BO_p5CN-CF', 'Y6BO_CB', 'Y6BO_p5CN-CB']
# selected_samples = ['Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB']
# selected_samples = ['Y6_CB', 'Y6_p5CN-CB']

# saxs_samples = ['PM6_CF', 'PM6_CB', 'PM6_CN']
saxs_samples = ['PM6_CF', 'PM6_CN']
# saxs_samples = ['Y6_CN', 'Y7_CN', 'Y7_CB']
saxs_sample_to_align = 'PM6_CN'
selected_qval = float(saxs_DS[saxs_sample_to_align].sel(q=shift_qval, method='nearest'))

# Shift all saxs to line up with selected sample qval
factor1 = selected_qval / float(saxs_DS['PM6_CB'].sel(q=shift_qval, method='nearest'))
factor2 = selected_qval / float(saxs_DS['PM6_CF'].sel(q=shift_qval, method='nearest'))
pm6_cb_shifted = saxs_DS['PM6_CB'] * factor1
pm6_cf_shifted = saxs_DS['PM6_CF'] * factor2

# factor1 = selected_qval / float(saxs_DS['Y7_CN'].sel(q=shift_qval, method='nearest'))
# factor2 = selected_qval / float(saxs_DS['Y7_CB'].sel(q=shift_qval, method='nearest'))
# y7_cn_shifted = saxs_DS['Y7_CN'] * factor1
# y7_cb_shifted = saxs_DS['Y7_CB'] * factor2

# saxs_colors = plt.cm.RdPu(np.linspace(0.4,1,len(saxs_samples)))
saxs_colors = plt.cm.Blues(np.linspace(0.5,1,len(saxs_samples)))
plt.close('all')

# for selected_samples in (['Y6_CF', 'Y6_p5CN-CF'],
#                          ['Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB'], 
#                          ['Y6_CB', 'Y6_p5CN-CB']):
# for selected_samples in (['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB'],
#                          ['PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF'],
#                          ['PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB']):
rsoxs_colors = plt.cm.Blues(np.linspace(0.3,0.8,len(selected_samples)))
fig, ax = plt.subplots(tight_layout=True, figsize=(4.8,4.3), dpi=130)
for i, sample_name in enumerate(tqdm(selected_samples)):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(DS, sample_name, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    DA = ar_DA.sel(energy=energy, method='nearest')

    # Shift RSoXS to line up with SAXS
    rsoxs_qval = float(DA.sel(q=shift_qval, method='nearest'))
    rsoxs_shift_factor = selected_qval / rsoxs_qval
    DA = DA * rsoxs_shift_factor

    # Plot rsoxs diff profile:
    DA.plot.line(ax=ax, color=rsoxs_colors[i], label=f'Film RSoXS: {sn[sample_name]}', 
                 linestyle='None', marker='.', markersize=3)

pm6_cf_shifted.plot.line(ax=ax, color=saxs_colors[0], label='Solution SAXS: PM6 in CF')    
# pm6_cb_shifted.plot.line(ax=ax, color=saxs_colors[1], label='Solution: PM6 in CB')
# y7_cn_shifted.plot.line(ax=ax, color=saxs_colors[1], label='Solution: Y7 in CN') 
# y7_cb_shifted.plot.line(ax=ax, color=saxs_colors[0], label='Solution: Y7 in CB') 
saxs_DS[saxs_sample_to_align].plot.line(ax=ax, color=saxs_colors[-1], label='Solution SAXS: PM6 in CN') 

ax.set(xscale='log', yscale='log', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
ax.set(title=f'PM6 film P-RSoXS and solution SAXS overlays \n RSoXS profile: $I_\parallel(Q, E) - I_\perp(Q, E)$, $E={energy}$ eV', ylim=(None, None), xlim=(None, 0.4))
ax.grid(visible=True, which='both', axis='x')
# ax.legend(title='Sample', loc=(1.02,0.2))
AA_str = '$Å^{-1}$'
legend = ax.legend(title=f'Data plotted:')
for handle in legend.legend_handles:
    handle.set_markersize(18)

plt.show()
# plt.close('all')

## Loop for saving many plots

In [ ]:
rsoxs_datasets['polar_oxygen']

In [ ]:
# make selection
edge = 'oxygen'
intensity_type = 'corr'
qlims = (0.01, 0.08)
chi_width = 30
energy_min = 525
energy_max = 545
cmin = 5e8
cmax = 5e10

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.data):
    for pol in [0, 90]:
        ### Select para & perp DataArrays
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
        
        ### ISI:
        # Slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')
        
        # Plot
        fig, ax = plt.subplots()
        para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='para', yscale='log')
        perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='perp', yscale='log')
        # para_ISI.plot.line(ax=ax, label='para', yscale='log')
        # perp_ISI.plot.line(ax=ax, label='perp', yscale='log')        
        fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
        ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
        ax.legend()
        fig.savefig(plotsPath.joinpath('isi_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')
        
        ### Linecut Maps:
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Intensity [arb. units]', labelpad=12)
        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        fig.set(tight_layout=True)
        axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')
        fig.savefig(plotsPath.joinpath('linecut_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

        ### AR Maps:
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        if para_DA.sample_name=='BareSiN':
            bareSiN_ar_DA = ar_DA.copy()
            
        # # Subtract AR from SiN:
        # ar_DA = ar_DA - bareSiN_ar_DA

        # Plot
        ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
        ax.figure.savefig(plotsPath.joinpath('ar_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

## Incorporating simulation output
old and incomplete

In [ ]:
import h5py

In [ ]:
whos PosixPath

In [ ]:
simsPath = rootPath.joinpath('imgs_analysis/sim_runs')

In [ ]:
[f.name for f in simPath.iterdir()]

In [ ]:
simPath = simsPath.joinpath('D1.0_a1.0_eps1.0_250pix_375size_0.5m_600steps_1')
h5path = simPath.joinpath('HDF5')
h5list = sorted(h5path.glob('E*h5'))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

#### The scattering pattern is contained within 'K0/projection.' If you simulated multiple incident angles there would be additional 'K#' keys. KIDList contains the vector values for each of the incident angles (propagation direction).

#### Instantiate the PyHyperScattering loader and integrator

In [ ]:
load = phs.load.cyrsoxsLoader(use_chunked_loading=True)
integ = phs.integrate.WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(simPath)

In [ ]:
raw

In [ ]:
remeshed = integ.integrateImageStack(raw)

In [ ]:
remeshed

In [ ]:
remeshed.sel(energy=285.1).plot()

In [ ]:
# c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=275).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=285).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=295).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.1,0.1),axes.set_ylim(-0.1,0.1)} for axes in ax]
plt.show()

In [ ]:
loaded_rsoxs.build_tiff_stack??